In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, models, utils

import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from sklearn import preprocessing
from PIL import Image
import time
import os
import copy
from collections import Counter

In [2]:
tabular_data = "../../data/SkidSteer_2019-08.csv"
color = pd.read_csv('../colorfulness/skid_steer_color_score.csv')

df = pd.read_csv(tabular_data, index_col=1)
df['Unique_ID'] = df[['Source','item#']].apply(lambda x: '_'.join(x),axis = 1)
df = df.filter(['Unique_ID','Winning Bid','Hours Final','Age at Sale (bin)','Bucket','Engine','Tires','Transmission'], axis = 1)
df = pd.merge(df, color,on='Unique_ID',how='inner')
df = df.rename(columns={
    'Unique_ID': "unique_id",
    'Hours Final': "hours_final",
    'Winning Bid': "winning_bid",
    'Age at Sale (bin)': "age_at_sale",
    'Bucket': "bucket",
    'Engine': "engine",
    'Tires': "tires",
    'Transmission': "transmission", 
    'socre': "colorfulness_score"
})
# color = pd.read_csv('skid_steer_color_score.csv')
# final_df = pd.merge(new_df, color,on='Unique_ID',how='inner')


### removal
# remove duplicant
duplicated_item = [item for item, count in Counter(df["unique_id"]).items() if count > 1]
df = df[~df['unique_id'].isin(duplicated_item)]

# remove not matched rows
image_item = [img_name.strip(".jpg") for img_name in os.listdir("../../data/images/")]
df = df[df["unique_id"].isin(image_item)]

# remove comma
df["winning_bid"] = df["winning_bid"].str.replace(',', '').astype(int)

# remove special image
df = df[df['unique_id'] != "rbauction_10525632"]

In [3]:
### winning_bid

# log-transform
df["winning_bid"] = np.log(df["winning_bid"])

# min max scale
mm_scaler_price = preprocessing.MinMaxScaler((-1, 1))
df["winning_bid"] = mm_scaler_price.fit_transform(df["winning_bid"].to_numpy().reshape(-1, 1))

In [4]:
### hours_final

# impute nan with median and new binary indicator
df["hours_final"] = df["hours_final"].str.replace(",", "")
df["hours_final"] = df["hours_final"].astype(float)
df.insert(3, column="hours_final_nan", value=df["hours_final"].isna().astype(int))
df.loc[df["hours_final"].isna(), "hours_final"] = df["hours_final"].median(skipna=True)

# log transform
df["hours_final"] = np.log(df["hours_final"])

# normalize
rb_scaler_hour = preprocessing.RobustScaler()
df["hours_final"] = rb_scaler_hour.fit_transform(np.array(df["hours_final"]).reshape(-1, 1))

In [5]:
### age_at_sale

# impute nan with median and new binary indicator
df["age_at_sale"] = df["age_at_sale"].astype(float)
df.insert(5, column="age_at_sale_nan", value=df["age_at_sale"].isna().astype(int))
df.loc[df["age_at_sale"].isna(), "age_at_sale"] = df["age_at_sale"].median(skipna=True)

# normalize
rb_scaler_age = preprocessing.RobustScaler()
df["age_at_sale"] = rb_scaler_age.fit_transform(np.array(df["age_at_sale"]).reshape(-1, 1))

In [6]:
### bucket
df.insert(7, column="bucket_bin", value=0)
df.loc[
    ~df["bucket"].isna() & 
    df["bucket"].str.contains("bucket", case=False) | 
    df["bucket"].str.contains("bkt", case=False), "bucket_bin"
] = 1

In [7]:
# shuffle and split
np.random.seed(1)
split = [0.7, 0.3]
split0 = round(df.shape[0] * split[0])
# split1 = round(df.shape[0] * (split[0] + split[1]))
df = df.sample(frac=1)
df_train = df.iloc[:split0]
df_val = df.iloc[split0:]

In [8]:
df_train.head()

,unique_id,winning_bid,hours_final,hours_final_nan,age_at_sale,age_at_sale_nan,bucket,bucket_bin,engine,tires,transmission,score
5364,ironplanet_1703726,0.105406,-6.732824,0,-0.666667,0,"74"" Wide General Purpose Smooth Edge Bucket",1,NaN,Cushion Tires,NaN,45.382919
3191,PW_DD1289,0.172815,-1.629216,0,-1.000000,0,"Kubota 68""W bucket",1,Kubota V3307-CR four cylinder turbo diesel engine,Titan 12-16.5 NHS tires,Two speed hydrostatic transmission,55.968483
5495,rbauction_10239624,0.219707,-0.164164,0,-0.666667,0,hyd Q/C bkt,1,NaN,NaN,NaN,35.573711
2563,PW_H1380,-0.072566,-0.769452,0,-0.333333,0,"80""W bucket",1,"81 HP, Case 3.2L four cylinder turbo diesel en...",12-16.5 tires,Hydrostatic transmission,38.453826
2883,PW_J8873,-0.305592,-0.416323,0,-0.333333,0,NaN,0,John Deere 5030TT001 3.0L turbo diesel engine,12-16.5 tires,Two speed hydrostatic transmission,72.305753


In [9]:
df_val.head()

,unique_id,winning_bid,hours_final,hours_final_nan,age_at_sale,age_at_sale_nan,bucket,bucket_bin,engine,tires,transmission,score
4563,bigiron_EN9531,0.164709,-0.503074,0,-0.333333,0,"84"" Bucket",1,"4-Cyl Turbo Diesel Engine, 84 Hp","Some Tires Have Cuts (Pictured), Tires- 14-17....",NaN,40.801346
972,rbauction_10693873,-0.861828,-0.443163,0,1.333333,0,bkt,1,NaN,NaN,NaN,19.575992
4915,bigiron_BO0203,-0.180158,-0.226316,0,1.333333,0,"Buckets 12"", 64"" Bucket",1,58 Horse Power Isuzu Diesel Engine,"12-16.5 Tires, Spare Tire And Rim",Hydrostat Transmission,19.348823
5351,ironplanet_1891010,-0.402470,-6.732824,0,-0.333333,0,"72"" Wide General Purpose Smooth Edge Bucket",1,NaN,NaN,NaN,59.205629
2819,PW_J2936,-0.402470,-1.926828,0,-0.333333,0,NaN,0,"Case four cylinder turbo diesel engine, Non-op...",12-16.5 tires,Hydrostatic transmission,27.390935


In [10]:
df_train.to_csv("./SkidSteer_2019-08_clean_train.csv")
df_val.to_csv("./SkidSteer_2019-08_clean_val.csv")

In [11]:
class skidsteer_dataset(Dataset):
    """Corrosion Detection dataset."""

    def __init__(self, 
                 csv_file, 
                 img_root, 
                 transform=None):
        """
        Args:
            csv_file (string): Path to the csv file with annotations.
            img_root (string): Directory with all the images.
            transform (callable, optional): Optional transform to be applied on a sample (including augmentation).
        """
        self.csv_file = pd.read_csv(csv_file, index_col=0)
        self.img_root = img_root
        self.transform = transform

    def __len__(self):
        return len(self.csv_file)

    def __getitem__(self, idx):
        '''Return one data point with a PIL image and its label.'''
        img_dir = os.path.join(self.img_root, self.csv_file.iloc[idx, 0]) + ".jpg"
        price = self.csv_file.iloc[idx, 1]
        image = Image.open(img_dir)
        others = torch.tensor(self.csv_file.iloc[idx, [2, 3, 4, 5, 7, 11]])  # change this when new columns are added
        
        if self.transform:
            image = self.transform(image)
        sample = {'image': image, 'price': price, "others": others}
        return sample

In [12]:
def norm2price(tensor, min_max_scaler):
    array2d = tensor.to("cpu").data.numpy().reshape(-1, 1)
    return np.exp(min_max_scaler.inverse_transform(array2d))

def price_MAE(outputs, prices, min_max_scaler):
    outputs = norm2price(outputs, min_max_scaler)
    prices = norm2price(prices, min_max_scaler)
    mae = np.abs(outputs - prices)
    maep = np.abs(outputs - prices) / prices
    return mae.mean(), mae, maep

In [13]:
def train_model(model, model_LU, criterion, optimizer, scheduler, num_epochs=25):
    since = time.time()

    best_model_wts = copy.deepcopy(model.state_dict())
    best_epoch = None
    best_loss = float("Inf")
    best_mae = float("Inf")
    best_mae_list = None
    best_maep_list = None
    all_loss = {x: [] for x in ['train', 'val']}
    all_mae = {x: [] for x in ['train', 'val']}

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch + 1, num_epochs))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode
            running_loss = 0.0
            running_mae = 0.0
            running_mae_list = []
            running_maep_list = []
            
            # Iterate over data.
            for items in dataloaders[phase]:
                images = items["image"].to(device)
                prices = items["price"].to(device)
                others = items["others"].to(device)
                optimizer.zero_grad()
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = cat_net(model, model_LU, images, others).squeeze()
                    loss = criterion(outputs, prices)
                    mae, mae_np, maep_np = price_MAE(outputs, prices, mm_scaler_price)
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()
                running_loss += loss.item() * images.size(0)
                running_mae += mae * images.size(0)
                running_mae_list += list(mae_np.flatten())
                running_maep_list += list(maep_np.flatten())
            if phase == 'train':
                scheduler.step(running_mae)
            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_mae = running_mae / dataset_sizes[phase]
            all_loss[phase].append(epoch_loss)
            all_mae[phase].append(epoch_mae)
            print('{} Loss: {:.4f}'.format(phase, epoch_loss))
            print('{} MAE: {:.4f}'.format(phase, epoch_mae))
            
            # deep copy the model
            if phase == 'val' and epoch_loss < best_loss:
                best_epoch = epoch + 1
                best_loss = epoch_loss
                best_mae = epoch_mae
                best_mae_list = running_mae_list
                best_maep_list = running_maep_list
                best_model_wts = copy.deepcopy(model.state_dict())
        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
    print('Best val Loss: {:4f} at epoch {}'.format(best_loss, best_epoch))
    print('Best val MAE: {:4f} at epoch {}'.format(best_mae, best_epoch))

    # load best model weights
    print("\nLoad the model weights at the best epoch")
    model.load_state_dict(best_model_wts)
    return model, best_mae_list, all_loss, all_mae

In [14]:
CSV_FILE = {"train": "./SkidSteer_2019-08_clean_train.csv",
            "val": "./SkidSteer_2019-08_clean_val.csv"}
IMG_ROOT = "../../data/images/"
TRANSFORM = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

datasets = {x: skidsteer_dataset(csv_file=CSV_FILE[x],
                                 img_root=IMG_ROOT,
                                 transform=TRANSFORM[x])
            for x in ["train", "val"]}
dataloaders = {x: DataLoader(datasets[x], 
                             batch_size=16, 
                             shuffle=True, 
                             num_workers=4)
               for x in ["train", "val"]}
dataset_sizes = {x: len(datasets[x]) for x in ["train", "val"]}
num_tabular_features = len(datasets["train"][0]["others"])

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [15]:
def cat_net(model, model_LU, images, others):
    z = torch.cat((model(images), others), dim=1)
    return model_LU(z)

## 0. (2053, 32, 1) + Tanh

In [16]:
torch.manual_seed(0)

### ResNet18
model_ft = models.resnet152(pretrained=True)
num_ftrs = model_ft.fc.in_features
model_ft.fc = nn.Identity()

### Freeze and Fine Tuning
for param in model_ft.parameters():
    param.requires_grad = False
for param in model_ft.layer3.parameters():
    param.requires_grad = True
for param in model_ft.layer4.parameters():
    param.requires_grad = True
assert [param.requires_grad for param in model_ft.parameters()][-1]

### LU
model_LU = nn.Sequential(
    nn.Linear(num_ftrs + num_tabular_features, 32),
    nn.Tanh(),
    nn.Linear(32, 1)
)

### model setup
model_ft = model_ft.to(device)
model_LU = model_LU.to(device)
criterion = nn.MSELoss()
# Observe that all parameters are being optimized
optimizer_ft = optim.SGD(list(model_ft.parameters())+list(model_LU.parameters()), lr=0.001, momentum=0.9)
# Decay LR by a factor of 0.1 every 7 epochs
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=20, gamma=0.1)

In [17]:
model_res152, mae_list_0, all_loss_0, all_mae_0 = train_model(model_ft, model_LU, criterion, optimizer_ft, exp_lr_scheduler, num_epochs=100)

Epoch 1/100
----------
train Loss: 0.0920
train MAE: 4535.1636
val Loss: 0.0523
val MAE: 3356.4828

Epoch 2/100
----------
train Loss: 0.0679
train MAE: 3891.4323
val Loss: 0.0478
val MAE: 3215.8706

Epoch 3/100
----------
train Loss: 0.0594
train MAE: 3630.5844
val Loss: 0.0431
val MAE: 3053.4833

Epoch 4/100
----------
train Loss: 0.0560
train MAE: 3548.3563
val Loss: 0.0595
val MAE: 3654.5581

Epoch 5/100
----------
train Loss: 0.0537
train MAE: 3471.5338
val Loss: 0.0413
val MAE: 2977.5897

Epoch 6/100
----------
train Loss: 0.0490
train MAE: 3301.3261
val Loss: 0.0452
val MAE: 3130.2727

Epoch 7/100
----------
train Loss: 0.0492
train MAE: 3297.2635
val Loss: 0.0726
val MAE: 4219.8945

Epoch 8/100
----------
train Loss: 0.0473
train MAE: 3263.9279
val Loss: 0.0402
val MAE: 2901.6929

Epoch 9/100
----------
train Loss: 0.0468
train MAE: 3229.3841
val Loss: 0.0454
val MAE: 3134.3024

Epoch 10/100
----------
train Loss: 0.0453
train MAE: 3166.4574
val Loss: 0.0379
val MAE: 2834.4005


## 1. (2053, 32, 1) + Sigmoid

In [18]:
torch.manual_seed(0)

### ResNet18
model_ft = models.resnet152(pretrained=True)
num_ftrs = model_ft.fc.in_features
model_ft.fc = nn.Identity()

### Freeze and Fine Tuning
for param in model_ft.parameters():
    param.requires_grad = False
for param in model_ft.layer3.parameters():
    param.requires_grad = True
for param in model_ft.layer4.parameters():
    param.requires_grad = True
assert [param.requires_grad for param in model_ft.parameters()][-1]

### LU
model_LU = nn.Sequential(
    nn.Linear(num_ftrs + num_tabular_features, 32),
    nn.Sigmoid(),
    nn.Linear(32, 1)
)

### model setup
model_ft = model_ft.to(device)
model_LU = model_LU.to(device)
criterion = nn.MSELoss()
# Observe that all parameters are being optimized
optimizer_ft = optim.SGD(list(model_ft.parameters())+list(model_LU.parameters()), lr=0.001, momentum=0.9)
# Decay LR by a factor of 0.1 every 7 epochs
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=20, gamma=0.1)

In [19]:
model_res152, mae_list_1, all_loss_1, all_mae_1 = train_model(model_ft, model_LU, criterion, optimizer_ft, exp_lr_scheduler, num_epochs=100)

Epoch 1/100
----------
train Loss: 0.0992
train MAE: 4745.8374
val Loss: 0.0623
val MAE: 3734.8699

Epoch 2/100
----------
train Loss: 0.0687
train MAE: 3947.3478
val Loss: 0.0496
val MAE: 3259.8694

Epoch 3/100
----------
train Loss: 0.0606
train MAE: 3690.1719
val Loss: 0.0446
val MAE: 3095.4583

Epoch 4/100
----------
train Loss: 0.0574
train MAE: 3601.5153
val Loss: 0.0527
val MAE: 3368.2378

Epoch 5/100
----------
train Loss: 0.0557
train MAE: 3529.3085
val Loss: 0.0415
val MAE: 2977.0102

Epoch 6/100
----------
train Loss: 0.0504
train MAE: 3346.1993
val Loss: 0.0405
val MAE: 2966.5879

Epoch 7/100
----------
train Loss: 0.0500
train MAE: 3332.2103
val Loss: 0.0526
val MAE: 3406.8758

Epoch 8/100
----------
train Loss: 0.0474
train MAE: 3259.2268
val Loss: 0.0385
val MAE: 2867.6982

Epoch 9/100
----------
train Loss: 0.0474
train MAE: 3240.2886
val Loss: 0.0389
val MAE: 2898.8871

Epoch 10/100
----------
train Loss: 0.0463
train MAE: 3195.0982
val Loss: 0.0371
val MAE: 2820.2490


In [24]:
import pickle
pickle.dump(mae_list_0, open("mae_list_0", "wb"))
pickle.dump(all_loss_0, open("all_loss_0", "wb"))
pickle.dump(all_mae_0, open("all_mae_0", "wb"))
pickle.dump(mae_list_1, open("mae_list_1", "wb"))
pickle.dump(all_loss_1, open("all_loss_1", "wb"))
pickle.dump(all_mae_1, open("all_mae_1", "wb"))

## 2. (2053, 32, 1) + Tanh + scheduler

In [16]:
torch.manual_seed(0)

### ResNet18
model_ft = models.resnet152(pretrained=True)
num_ftrs = model_ft.fc.in_features
model_ft.fc = nn.Identity()

### Freeze and Fine Tuning
for param in model_ft.parameters():
    param.requires_grad = False
for param in model_ft.layer3.parameters():
    param.requires_grad = True
for param in model_ft.layer4.parameters():
    param.requires_grad = True
assert [param.requires_grad for param in model_ft.parameters()][-1]

### LU
model_LU = nn.Sequential(
    nn.Linear(num_ftrs + num_tabular_features, 32),
    nn.Tanh(),
    nn.Linear(32, 1)
)

### model setup
model_ft = model_ft.to(device)
model_LU = model_LU.to(device)
criterion = nn.MSELoss()
# Observe that all parameters are being optimized
optimizer_ft = optim.SGD(list(model_ft.parameters())+list(model_LU.parameters()), lr=0.001, momentum=0.9)
# Decay LR by a factor of 0.1 every 7 epochs
exp_lr_scheduler = lr_scheduler.ReduceLROnPlateau(optimizer_ft, factor=0.1, verbose=True)

In [17]:
model_res152, mae_list_2, all_loss_2, all_mae_2 = train_model(model_ft, model_LU, criterion, optimizer_ft, exp_lr_scheduler, num_epochs=100)

Epoch 1/100
----------
train Loss: 0.0920
train MAE: 4535.2175
val Loss: 0.0523
val MAE: 3356.4244

Epoch 2/100
----------
train Loss: 0.0679
train MAE: 3891.6452
val Loss: 0.0478
val MAE: 3215.2317

Epoch 3/100
----------
train Loss: 0.0594
train MAE: 3630.3096
val Loss: 0.0431
val MAE: 3053.4651

Epoch 4/100
----------
train Loss: 0.0560
train MAE: 3548.2751
val Loss: 0.0595
val MAE: 3655.6614

Epoch 5/100
----------
train Loss: 0.0537
train MAE: 3471.3711
val Loss: 0.0413
val MAE: 2975.7702

Epoch 6/100
----------
train Loss: 0.0490
train MAE: 3300.9433
val Loss: 0.0453
val MAE: 3132.5130

Epoch 7/100
----------
train Loss: 0.0492
train MAE: 3296.1072
val Loss: 0.0727
val MAE: 4224.1736

Epoch 8/100
----------
train Loss: 0.0473
train MAE: 3263.1697
val Loss: 0.0402
val MAE: 2901.1215

Epoch 9/100
----------
train Loss: 0.0467
train MAE: 3228.9887
val Loss: 0.0455
val MAE: 3137.2490

Epoch 10/100
----------
train Loss: 0.0453
train MAE: 3166.0073
val Loss: 0.0379
val MAE: 2834.8794


## 3. (2053, 32, 1) + Tanh + scheduler

In [18]:
torch.manual_seed(0)

### ResNet18
model_ft = models.resnet152(pretrained=True)
num_ftrs = model_ft.fc.in_features
model_ft.fc = nn.Identity()

### Freeze and Fine Tuning
for param in model_ft.parameters():
    param.requires_grad = False
for param in model_ft.layer3.parameters():
    param.requires_grad = True
for param in model_ft.layer4.parameters():
    param.requires_grad = True
assert [param.requires_grad for param in model_ft.parameters()][-1]

### LU
model_LU = nn.Sequential(
    nn.Linear(num_ftrs + num_tabular_features, 32),
    nn.Tanh(),
    nn.Linear(32, 1)
)

### model setup
model_ft = model_ft.to(device)
model_LU = model_LU.to(device)
criterion = nn.MSELoss()
# Observe that all parameters are being optimized
optimizer_ft = optim.SGD(list(model_ft.parameters())+list(model_LU.parameters()), lr=0.001, momentum=0.9)
# Decay LR by a factor of 0.1 every 7 epochs
exp_lr_scheduler = lr_scheduler.ReduceLROnPlateau(optimizer_ft, factor=0.1, verbose=True)

In [19]:
model_res152, mae_list_3, all_loss_3, all_mae_3 = train_model(model_ft, model_LU, criterion, optimizer_ft, exp_lr_scheduler, num_epochs=200)

Epoch 1/200
----------
train Loss: 0.0920
train MAE: 4535.2421
val Loss: 0.0523
val MAE: 3356.5317

Epoch 2/200
----------
train Loss: 0.0679
train MAE: 3891.7404
val Loss: 0.0478
val MAE: 3216.1251

Epoch 3/200
----------
train Loss: 0.0594
train MAE: 3630.5861
val Loss: 0.0431
val MAE: 3053.6034

Epoch 4/200
----------
train Loss: 0.0560
train MAE: 3548.3931
val Loss: 0.0595
val MAE: 3654.5785

Epoch 5/200
----------
train Loss: 0.0537
train MAE: 3471.4900
val Loss: 0.0413
val MAE: 2977.3775

Epoch 6/200
----------
train Loss: 0.0490
train MAE: 3301.3993
val Loss: 0.0452
val MAE: 3131.4367

Epoch 7/200
----------
train Loss: 0.0492
train MAE: 3296.0732
val Loss: 0.0728
val MAE: 4225.8910

Epoch 8/200
----------
train Loss: 0.0473
train MAE: 3263.1863
val Loss: 0.0402
val MAE: 2902.3317

Epoch 9/200
----------
train Loss: 0.0468
train MAE: 3229.6498
val Loss: 0.0455
val MAE: 3138.5456

Epoch 10/200
----------
train Loss: 0.0453
train MAE: 3165.9938
val Loss: 0.0379
val MAE: 2833.5303


In [20]:
import pickle
pickle.dump(mae_list_2, open("mae_list_2", "wb"))
pickle.dump(all_loss_2, open("all_loss_2", "wb"))
pickle.dump(all_mae_2, open("all_mae_2", "wb"))
pickle.dump(mae_list_3, open("mae_list_3", "wb"))
pickle.dump(all_loss_3, open("all_loss_3", "wb"))
pickle.dump(all_mae_3, open("all_mae_3", "wb"))